Importing packages needed

In [ ]:
import numpy as np #for working with arrays
import pandas as pd #for data manipulation and analysis
import matplotlib.pyplot as plt #for plots and visualisation
import seaborn as sns #for visualisation

Importing data as a Dataframe

In [ ]:
#data is a csv file on local desktop. File has been included in zip file for reproducibility
#data = pd.read_csv()

#data is online - to improve reproducibility and scalability
#from urllib.request import urlretrieve
#url='' # assign file url
#urlretrieve(url,'') #save file locally
#data = pd.read_csv(, sep='') Read file into a DataFrame - sep = delimeter in file

Checking and cleaning the data

In [ ]:
#data.head(-1) #this will show the first and last 5 rows as a quick check to see if the data was loaded properly
#data.info #this will show the column names, data types and also check if there is missing values and content.
#data.decribe # this will showw summary statistics

This could have been done with the below as well:
data.shape() #this will show the number of rows and column in our dataframe
data.column #this will show the column names
data.isnull().sum() #this will count and check for missing values in the data. Without .sum() would return Boolean array, which is harder to see especially with large data
